<a href="https://colab.research.google.com/github/SIDHARTH06/AI_CLUB_LOGO_GEN_GAN/blob/main/AIproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import time
from google.colab import drive
import h5py
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


11493376/11490434 [==============================] - 0s 0us/step


In [10]:
hdf5_path="/content/gdrive/My Drive/LLD-icon.hdf5"
dataset=h5py.File(hdf5_path,"r")